# Text Summarizer

## Overview
This is a text summarizer based loosely on [SMMRY](http://smmry.com/). It returns the most important sentences in a document. It works by assigning a score to each sentence in one of 2 ways:

1. summing the tf-idf values of its constituent words OR

2. calculating the distance of each sentence from all the other sentences.

<!-- TEASER_END -->

Sentences are then ranked and the top-scoring ones are returned in chronological order.

In [1]:
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.spatial.distance import pdist, squareform
import numpy as np

def summarize(text, p, scoring='sum'):
    '''
    Summarizes a text by returning only the most important sentences
    
    Inputs:
        text - str - the text to be summarized
        p - int or float, the number of sentences to be returned or, if p < 1, the percentage of sentences
        scoring - str - scoring method. To sum tf-idf values use 'sum'. To measure sentence-vector distance, 
                        use one of the following metrics:
                            ‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘cityblock’, ‘correlation’,
                            ‘cosine’, ‘dice’, ‘euclidean’, ‘hamming’, ‘jaccard’, ‘kulsinski’,
                            ‘mahalanobis’, ‘matching’, ‘minkowski’, ‘rogerstanimoto’, ‘russellrao’,
                            ‘seuclidean’, ‘sokalmichener’, ‘sokalsneath’, ‘sqeuclidean’, ‘yule’

    Output: str - the summarized text
    '''
    
    # create blob, determine number of sentences to return
    blob = TextBlob(text)
    if p < 1:
        p = int(round(p*len(blob.sentences)))
    
    
    scores = []
    if scoring == 'sum':

        # create tf-idf matrix
        vec = TfidfVectorizer(stop_words='english')
        dtm = vec.fit_transform((str(x) for x in blob.sentences))
        
        # calculate and store scores
        for i, c in enumerate(dtm):
            scores.append((i, c.sum()))
    
    else:
        # create count matrix for distance method
        vec = CountVectorizer(stop_words='english')
        dtm = vec.fit_transform((str(x) for x in blob.sentences))
        
        # calculate pair-wise distances between each sentence
        dtm = dtm.toarray()
        y = pdist(dtm, scoring)
        
        # change sparse representation into a dense matrix
        y = squareform(y)
        
        # calcualte and store scores
        for i, c in enumerate(y):
            scores.append((i, np.nanmean(c)))

    # sort sentences by score, select the top p sentences, then re-order them chronologically
    top_sentences = sorted(scores, key=lambda x: x[1], reverse=True)
    summary = top_sentences[:p]
    summary = sorted(summary)
    
    result = TextBlob('')
    for i in summary:
        result += blob.sentences[i[0]]
    return result

## Example

In [2]:
with open('../data/little_red_riding_hood.txt') as f:
    story = f.read()
    
def percent_of_story(summary):
    return len(summary)/float(len(story))

In [3]:
tfidf = summarize(story, .4)
percent_of_story(tfidf)

0.6689223697650664

In [4]:
euclid = summarize(story, .4, scoring='euclidean')
percent_of_story(euclid)

0.666879468845761

In [5]:
cheb = summarize(story, .7, scoring='chebyshev')
percent_of_story(cheb)

0.6363636363636364

## Sample Output (Euclidean Distance)

Once upon a time there was a dear little girl who was loved by everyone who looked at her, but most of all by her grandmother, and there was nothing that she would not have given to the child.

Once she gave her a little riding hood of red velvet, which suited her so well that she would never wear anything else; so she was always called 'Little Red Riding Hood.'

One day her mother said to her: 'Come, Little Red Riding Hood, here is a piece of cake and a bottle of wine; take them to your grandmother, she is ill and weak, and they will do her good.Set out before it gets hot, and when you are going, walk nicely and quietly and do not run off the path, or you may fall and break the bottle, and then your grandmother will get nothing; and when you go into her room, don't forget to say, "Good morning", and don't peep into every corner before you do it.'

The grandmother lived out in the wood, half a league from the village, and just as Little Red Riding Hood entered the wood, a wolf met her.

'Cake and wine; yesterday was baking-day, so poor sick grandmother is to have something good, to make her stronger.'

'A good quarter of a league farther on in the wood; her house stands under the three large oak-trees, the nut-trees are just below; you surely must know it,' replied Little Red Riding Hood.

So he walked for a short time by the side of Little Red Riding Hood, and then he said: 'See, Little Red Riding Hood, how pretty the flowers are about here - why do you not look round? I believe, too, that you do not hear how sweetly the little birds are singing; you walk gravely along as if you were going to school, while everything else out here in the wood is merry.'

Little Red Riding Hood raised her eyes, and when she saw the sunbeams dancing here and there through the trees, and pretty flowers growing everywhere, she thought: 'Suppose I take grandmother a fresh nosegay; that would please her too.'

And whenever she had picked one, she fancied that she saw a still prettier one farther on, and ran after it, and so got deeper and deeper into the wood.

The wolf lifted the latch, the door sprang open, and without saying a word he went straight to the grandmother's bed, and devoured her.

Little Red Riding Hood, however, had been running about picking flowers, and when she had gathered so many that she could carry no more, she remembered her grandmother, and set out on the way to her.She was surprised to find the cottage-door standing open, and when she went into the room, she had such a strange feeling that she said to herself: 'Oh dear!' 

She called out: 'Good morning,' but received no answer; so she went to the bed and drew back the curtains.

There lay her grandmother with her cap pulled far over her face, and looking very strange.

When the wolf had appeased his appetite, he lay down again in the bed, fell asleep and began to snore very loud.

The huntsman was just passing the house, and thought to himself: 'How the old woman is snoring!' 

But just as he was going to fire at him, it occurred to him that the wolf might have devoured the grandmother, and that she might still be saved, so he did not fire, but took a pair of scissors, and began to cut open the stomach of the sleeping wolf.

When he had made two snips, he saw the little red riding hood shining, and then he made two snips more, and the little girl sprang out, crying: 'Ah, how frightened I have been!'

Red Riding Hood, however, quickly fetched great stones with which they filled the wolf's belly, and when he awoke, he wanted to run away, but the stones were so heavy that he collapsed at once, and fell dead.

The huntsman drew off the wolf's skin and went home with it; the grandmother ate the cake and drank the wine which Red Riding Hood had brought, and revived.

But Red Riding Hood thought to herself: 'As long as I live, I will never leave the path by myself to run into the wood, when my mother has forbidden me to do so.'

It is also related that once, when Red Riding Hood was again taking cakes to the old grandmother, another wolf spoke to her, and tried to entice her from the path.

Red Riding Hood, however, was on her guard, and went straight forward on her way, and told her grandmother that she had met the wolf, and that he had said 'good morning' to her, but with such a wicked look in his eyes, that if they had not been on the public road she was certain he would have eaten her up.

Soon afterwards the wolf knocked, and cried: 'Open the door, grandmother, I am Little Red Riding Hood, and am bringing you some cakes.'

But they did not speak, or open the door, so the grey-beard stole twice or thrice round the house, and at last jumped on the roof, intending to wait until Red Riding Hood went home in the evening, and then to steal after her and devour her in the darkness.

In front of the house was a great stone trough, so she said to the child: 'Take the pail, Red Riding Hood; I made some sausages yesterday, so carry the water in which I boiled them to the trough.'

Then the smell of the sausages reached the wolf, and he sniffed and peeped down, and at last stretched out his neck so far that he could no longer keep his footing and began to slip, and slipped down from the roof straight into the great trough, and was drowned.

## Results and thoughts

Even with one third removed, the story is fairly easy to follow. The summarizer dealt best with exposition and narration and all the major plot-points are covered. The summarizer doesn't do so well with dialogue, however, which makes sense, since deleting any portion of a dialogue interrupts the flow of the conversation.

The tf-idf sum method is heavily biased towards long sentences. I tried to temper this by assigning the *average* tf-idf as the score instead of the sum, but this led to a summary comprised almost entirely of short bits of disconnected dialogue. Distance methods are a bit shorter, but not much. Different distances return very different summaries, so this is one way to tune the method to specific types of documents.

SMMRY mentions that it removes unnecessary clauses. Incorporating something like this could also substantially shorten the final summary. One final idea for further improvement is automating the amount of material that is returned. This seems like it might be quite difficult, since a human is needed to evaluate the quality of the summary.